In [1]:
import sys, os
sys.path.insert(1, os.path.join(sys.path[0], ".."))
from modt.modt import MoDT
from modt._initialization import *
from modt.visualization import *
from modt.utility import *

import pickle
from timeit import default_timer as timer

import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import normalize
from sklearn.model_selection import RepeatedKFold

In [2]:
from IPython.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_colwidth', None)

In [3]:
df = pd.read_pickle("dataframes/ex4a_df_intersections_feature_selection.pd")

In [4]:
df

,dataset,DT-LDA,DT-LDAm,DT-LR,DT-LRm,DT-XGB,DT-L,LDA-LDAm,LDA-LR,LDA-LRm,...,LDAm-LR,LDAm-LRm,LDAm-XGB,LDAm-L,LR-LRm,LR-XGB,LR-L,LRm-XGB,LRm-L,XGB-L
0,abalone_input.pd,0.5,0.5,0.5,0.5,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,0.5,0.5,1.0,0.5,0.5,0.5,0.5,0.0
1,adult_input.pd,0.5,0.5,0.5,0.5,0.5,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
2,banknote_input.pd,0.5,0.5,1.0,1.0,1.0,0.0,1.0,0.5,0.5,...,0.5,0.5,0.5,0.5,1.0,1.0,0.0,1.0,0.0,0.0
3,bank_input.pd,0.5,0.5,0.5,0.5,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,0.5,0.0,1.0,0.5,0.0,0.5,0.0,0.0
4,breast_cancer_input.np,0.0,0.0,1.0,1.0,0.5,0.5,1.0,0.0,0.0,...,0.0,0.0,0.5,0.0,1.0,0.5,0.5,0.5,0.5,0.5
5,cars_input.pd,1.0,0.5,1.0,1.0,1.0,0.5,0.5,1.0,1.0,...,0.5,0.5,0.5,0.5,1.0,1.0,0.5,1.0,0.5,0.5
6,contraceptive_input.pd,1.0,1.0,1.0,1.0,0.5,0.0,1.0,1.0,1.0,...,1.0,1.0,0.5,0.0,1.0,0.5,0.0,0.5,0.0,0.0
7,generated6_input.np,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,hrss_input.pd,0.0,0.0,0.0,0.0,0.5,0.5,1.0,1.0,1.0,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.5
9,iris_input.pd,1.0,1.0,1.0,1.0,1.0,0.5,1.0,1.0,1.0,...,1.0,1.0,1.0,0.5,1.0,1.0,0.5,1.0,0.5,0.5


In [5]:
df.mean()

DT-LDA      0.541667
DT-LDAm     0.500000
DT-LR       0.666667
DT-LRm      0.666667
DT-XGB      0.750000
DT-L        0.250000
LDA-LDAm    0.958333
LDA-LR      0.791667
LDA-LRm     0.791667
LDA-XGB     0.625000
LDA-L       0.250000
LDAm-LR     0.750000
LDAm-LRm    0.750000
LDAm-XGB    0.583333
LDAm-L      0.250000
LR-LRm      0.916667
LR-XGB      0.666667
LR-L        0.291667
LRm-XGB     0.666667
LRm-L       0.250000
XGB-L       0.250000
dtype: float64

In [6]:
methods = ["DT","LDA","LDAm","LR","LRm","XGB","L"]

In [7]:
matrix = pd.DataFrame(0.0, index=methods, columns=methods)
matrix

,DT,LDA,LDAm,LR,LRm,XGB,L
DT,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LDA,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LDAm,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LR,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LRm,0.0,0.0,0.0,0.0,0.0,0.0,0.0
XGB,0.0,0.0,0.0,0.0,0.0,0.0,0.0
L,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
matrix["DT"]["LDA"] =  0.541667
matrix["DT"]["LDAm"] =  0.500000
matrix["DT"]["LR"] =  0.666667
matrix["DT"]["LRm"] =  0.666667
matrix["DT"]["XGB"] =  0.750000
matrix["DT"]["L"] =  0.250000
matrix["LDA"]["LDAm"] =  0.958333
matrix["LDA"]["LR"] =  0.791667
matrix["LDA"]["LRm"] =  0.791667
matrix["LDA"]["XGB"] =  0.625000
matrix["LDA"]["L"] =  0.250000
matrix["LDAm"]["LR"] =  0.750000
matrix["LDAm"]["LRm"] =  0.750000
matrix["LDAm"]["XGB"] =  0.583333
matrix["LDAm"]["L"] =  0.250000
matrix["LR"]["LRm"] =  0.916667
matrix["LR"]["XGB"] =  0.666667
matrix["LR"]["L"] =  0.291667
matrix["LRm"]["XGB"] =  0.666667
matrix["LRm"]["L"] =  0.250000
matrix["XGB"]["L"] =  0.250000

In [9]:
matrix = matrix + matrix.T

In [10]:
matrix

,DT,LDA,LDAm,LR,LRm,XGB,L
DT,0.000000,0.541667,0.500000,0.666667,0.666667,0.750000,0.250000
LDA,0.541667,0.000000,0.958333,0.791667,0.791667,0.625000,0.250000
LDAm,0.500000,0.958333,0.000000,0.750000,0.750000,0.583333,0.250000
LR,0.666667,0.791667,0.750000,0.000000,0.916667,0.666667,0.291667
LRm,0.666667,0.791667,0.750000,0.916667,0.000000,0.666667,0.250000
XGB,0.750000,0.625000,0.583333,0.666667,0.666667,0.000000,0.250000
L,0.250000,0.250000,0.250000,0.291667,0.250000,0.250000,0.000000


In [11]:
def printer(n):
    if n == 0.0:
        return " "
    else:
        return "%.2f" % round(n,2)

In [12]:
for index, row in matrix.iterrows():
    row_string = "& "
    for element in row:
        row_string += printer(element)
        row_string += " & "
    row_string = row_string[:-3]
    row_string += "\\\\"
    print(row_string)

&   & 0.54 & 0.50 & 0.67 & 0.67 & 0.75 & 0.25\\
& 0.54 &   & 0.96 & 0.79 & 0.79 & 0.62 & 0.25\\
& 0.50 & 0.96 &   & 0.75 & 0.75 & 0.58 & 0.25\\
& 0.67 & 0.79 & 0.75 &   & 0.92 & 0.67 & 0.29\\
& 0.67 & 0.79 & 0.75 & 0.92 &   & 0.67 & 0.25\\
& 0.75 & 0.62 & 0.58 & 0.67 & 0.67 &   & 0.25\\
& 0.25 & 0.25 & 0.25 & 0.29 & 0.25 & 0.25 &  \\
